In [6]:
!pip install langchain langchain-community pypdf bs4 pymupdf wikipedia langchain-text-splitters  langchain-openai chromadb arxiv wikipedia langchain_chroma langchain-core sentence-transformers langchain-huggingface faiss-cpu python-dotenv langchain-groq fastapi uvicorn langserve sse_starlette python-dotenv langchain-groq langchain-community langchain-core bs4 langchain-text-splitters langchain_chroma langchain-huggingface langchain-groq streamlit langchain-openai faiss-cpu mysql-connector-python validators==0.28.1 youtube_transcript_api
!pip install -U duckduckgo-search unstructured pytube
!npm install localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00

up to date, audited 23 packages in 494ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [9]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.196.120.83
your url is: https://social-roses-peel.loca.lt


In [10]:
import validators
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import YoutubeLoader, UnstructuredURLLoader
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="Gemma-7b-It", groq_api_key=groq_api_key)

In [11]:
prompt_template="""
Provide summary of the following content in 300 words:
Content:{text}
"""

prompt=PromptTemplate(template=prompt_template, input_variables=["text"])

In [13]:
generic_url= "https://www.youtube.com/watch?v=zupyoDbnwDQ"

In [14]:
if "youtube.com" in generic_url:
  loader=YoutubeLoader.from_youtube_url(generic_url, add_video_info=True)
else:
  loader=UnstructuredURLLoader(urls=[generic_url], ssl_verify=False,
  headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"})
docs=loader.load()

In [15]:
docs

[Document(metadata={'source': 'zupyoDbnwDQ', 'title': 'Avoid Vaisnav Aparadh by P.L.U.S. & J.O.K.E.B.A.D. formula |ISKCON SJM Bangalore| Radheshyam Das', 'description': 'Unknown', 'view_count': 2599, 'thumbnail_url': 'https://i.ytimg.com/vi/zupyoDbnwDQ/hq720.jpg?v=66e3e1e3', 'publish_date': '2024-09-12 00:00:00', 'length': 4760, 'author': 'Radheshyam Das Devotional videos'}, page_content="go Gir [Music] [Applause] [Music] [Applause] [Applause] [Music] [Applause] y m Gana [Music] [Applause] [Music] [Applause] [Music] [Applause] [Applause] Krishna Krishna Krishna Krishna h h h [Music] [Music] [Applause] [Music] Krishna Krishna [Applause] H MRA jagat balad subadra Maharani Kang bhat so you can take the verse 2.3.4 oh [Music] [Music] Krishna [Music] Nam [Music] H Krishna H Krishna Krishna Krishna h h h h h om om [Music] [Music] [Music] Kumar It's Not Duplicate it's not going or you can take in your own mobile otherwise H lose contact [Applause] [Music] t [Music] har proes [Music] [Music] T

In [17]:
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len
)
docs=text_splitter.split_documents(docs)
docs

[Document(metadata={'source': 'zupyoDbnwDQ', 'title': 'Avoid Vaisnav Aparadh by P.L.U.S. & J.O.K.E.B.A.D. formula |ISKCON SJM Bangalore| Radheshyam Das', 'description': 'Unknown', 'view_count': 2599, 'thumbnail_url': 'https://i.ytimg.com/vi/zupyoDbnwDQ/hq720.jpg?v=66e3e1e3', 'publish_date': '2024-09-12 00:00:00', 'length': 4760, 'author': 'Radheshyam Das Devotional videos'}, page_content="go Gir [Music] [Applause] [Music] [Applause] [Applause] [Music] [Applause] y m Gana [Music] [Applause] [Music] [Applause] [Music] [Applause] [Applause] Krishna Krishna Krishna Krishna h h h [Music] [Music] [Applause] [Music] Krishna Krishna [Applause] H MRA jagat balad subadra Maharani Kang bhat so you can take the verse 2.3.4 oh [Music] [Music] Krishna [Music] Nam [Music] H Krishna H Krishna Krishna Krishna h h h h h om om [Music] [Music] [Music] Kumar It's Not Duplicate it's not going or you can take in your own mobile otherwise H lose contact [Applause] [Music] t [Music] har proes [Music] [Music] T

In [18]:
chunks_prompt="""
Write a concise and short summary of the following speech:
Speech:{text}
Summary:
"""

map_prompt_template=PromptTemplate(
    input_variables=["text"],
    template=chunks_prompt
)

In [19]:
final_prompt="""
Provide hte final summary with these important points.
Add a motivational title and start a precise summary with an introduction nad provide the summary in number points.
Speech:{text}
"""

final_prompt_template=PromptTemplate(
    input_variables=["text"],
    template=final_prompt
)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide hte final summary with these important points.\nAdd a motivational title and start a precise summary with an introduction nad provide the summary in number points.\nSpeech:{text}\n')

In [20]:
chain=load_summarize_chain(llm, chain_type="map_reduce", map_prompt=map_prompt_template, combine_prompt=final_prompt_template, verbose=True)
output_summary=chain.run(docs)
output_summary

Prompt after formatting:

Write a concise and short summary of the following speech:
Speech:go Gir [Music] [Applause] [Music] [Applause] [Applause] [Music] [Applause] y m Gana [Music] [Applause] [Music] [Applause] [Music] [Applause] [Applause] Krishna Krishna Krishna Krishna h h h [Music] [Music] [Applause] [Music] Krishna Krishna [Applause] H MRA jagat balad subadra Maharani Kang bhat so you can take the verse 2.3.4 oh [Music] [Music] Krishna [Music] Nam [Music] H Krishna H Krishna Krishna Krishna h h h h h om om [Music] [Music] [Music] Kumar It's Not Duplicate it's not going or you can take in your own mobile otherwise H lose contact [Applause] [Music] t [Music] har proes [Music] [Music] Together Al together please [Music] [Applause] yeah thank you transl by divine grace certainly that heart is still framed which in spite of once chanting the holy name of the Lord with concentration does not change even when one displays such signs of transformation has tears in the eyes an ecstatic 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2429 > 1024). Running this sequence through the model will result in indexing errors


Prompt after formatting:

Provide hte final summary with these important points.
Add a motivational title and start a precise summary with an introduction nad provide the summary in number points.
Speech:The speech emphasizes the transformative power of chanting the Hare Krishna mantra, focusing on the change it brings to the heart. It highlights the importance of humility and service to the Lord, leading to detachment from material desires and an increase in loving devotion.

The speech emphasizes the importance of humility and service in spiritual practice. It compares the pursuit of wealth or material possessions to inappropriate uses of tools. True happiness and fulfillment come from recognizing Krishna as the ultimate master and serving him with joy. The speaker encourages embracing the role of a servant, similar to the joyful contentment of a child in the presence of a loving parent.

The speech emphasizes the importance of welcoming Lord Krishna into the heart and engaging in re

"## Speech Summary: The Power of Chanting and Serving the Lord\n\n**Introduction:**\n\nThis speech emphasizes the transformative power of chanting the Hare Krishna mantra and the importance of humility and service in spiritual practice.\n\n**Key Points:**\n\n**1. Importance of Humility and Service:**\n- Chanting and serving the Lord should be performed with humility and selfless devotion.\n- True happiness comes from recognizing Krishna as the ultimate master and serving him with joy.\n\n**2. Engaging in Rituals:**\n- Regularly performing devotional practices like chanting, studying scriptures, and serving others purifies the heart and leads to spiritual growth.\n- Offering respect and reverence through rituals like offering oblations is crucial.\n\n**3. Avoiding Offenses:**\n- Distinguish between **offense of omission** (neglecting duties) and **offense of commission** (doing inappropriate things).\n- Approach others with respect, offering support and understanding rather than critici

In [36]:
def summarizer(generic_url, llm, prompt, chunks_prompt, final_prompt):
  if "youtube.com" in generic_url:
    loader=YoutubeLoader.from_youtube_url(generic_url, add_video_info=True)
  else:
    loader = WebBaseLoader(generic_url)
  docs=loader.load()
  text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len)
  docs=text_splitter.split_documents(docs)
  map_prompt_template=PromptTemplate(
    input_variables=["text"],
    template=chunks_prompt
  )
  final_prompt_template=PromptTemplate(
    input_variables=["text"],
    template=final_prompt
  )
  chain=load_summarize_chain(llm, chain_type="map_reduce", map_prompt=map_prompt_template, combine_prompt=final_prompt_template, verbose=False)
  output_summary=chain.run(docs)
  return output_summary

In [26]:
generic_url="https://www.youtube.com/watch?v=zupyoDbnwDQ"
summarizer(generic_url, llm, prompt, chunks_prompt, final_prompt)

'## **Transformed Hearts Through Chanting: A Summary**\n\nThis speech emphasizes the transformative power of chanting the holy name of Lord Krishna on the heart. It highlights the importance of understanding our humility before God and welcoming him into our hearts for true happiness.\n\n**Key Points:**\n\n**1. Spiritual Growth and Detachment:**\n- Chanting gradually detaches us from material desires.\n- True spiritual growth involves recognizing our humility before God.\n\n**2. Relationship with Krishna:**\n- Chasing worldly desires is like misusing tools.\n- Following Krishna brings real fulfillment and joy.\n- Serving the Lord brings joy and peace.\n\n**3. Chanting and Meditation:**\n- Chanting with awareness and following principles leads to spiritual growth.\n- Symptoms of ecstasy include physical and emotional changes.\n- Both external and internal transformation are crucial.\n\n**4. Avoiding Offenses:**\n- Two types of offenses: offense of omission (neglecting respect) and offen

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
generic_url="https://python.langchain.com/v0.1/docs/modules/agents/agent_types/"
if "youtube.com" in generic_url:
  loader=YoutubeLoader.from_youtube_url(generic_url, add_video_info=True)
else:
  loader=UnstructuredURLLoader(urls=[generic_url], ssl_verify=False,
  headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"})
docs=loader.load()

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'python.langchain.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
ERROR:langchain_community.document_loaders.url:Error fetching or processing https://python.langchain.com/v0.1/docs/modules/agents/agent_types/, exception: No such file or directory: '/root/nltk_data/tokenizers/punkt/PY3_tab'


In [32]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://pritilakshanam.wordpress.com/2024/09/03/in-want-of-peace/")
docs=loader.load()

In [33]:
docs

[Document(metadata={'source': 'https://pritilakshanam.wordpress.com/2024/09/03/in-want-of-peace/', 'title': 'In want of Peace – Prīti Lakṣaṇam', 'description': 'If men want to live in harmony and peace then they must accept Krishna Consciousness because they cannot achieve the highest standard if they are living in the bodily concept of the life.', 'language': 'en'}, page_content='\n\n\n\n\n\nIn want of Peace – Prīti Lakṣaṇam\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConnect with me\n\n\n\nPrīti Lakṣaṇam\nSearch\n\n\n\n\n\n\n\n\nBlog PostsShort Lessons\n\n\n\n\n\n\n\n\n\n\n\nLong Post\nIn want of\xa0Peace\n\nPublished by\nThe Bhagavad Dharma\non\nSeptember 3, 2024\n\n\n\n\n\nIf men want to live in harmony and peace then they must accept Krishna Consciousness because they cannot achieve the highest standard if they are living in the bod

In [37]:
generic_url="https://pritilakshanam.wordpress.com/2024/09/03/in-want-of-peace/"
summarizer(generic_url, llm, prompt, chunks_prompt, final_prompt)

'## **Finding Peace Through Spiritual Connection: A Speech Summary**\n\n**Introduction:**\n\nThis speech emphasizes the crucial role of accepting Krishna Consciousness in achieving lasting peace and harmony in our lives. By transcending the limitations of our bodily consciousness, we can attain the ultimate goal of spiritual liberation.\n\n**Key Points:**\n\n1. **Transcending Narrow-mindedness:** The speech highlights the widespread conflict and suffering in the modern world, attributing it to narrow-mindedness, selfishness, and a materialistic mindset. It argues that true peace can only be achieved by recognizing our temporary existence on Earth and cultivating spiritual awareness beyond material possessions.\n\n\n2. **Community and Humility:** The importance of community and humility for achieving peace and harmony is emphasized. Pride and ego are identified as obstacles to creating a harmonious society. By acknowledging the value of others, accepting differences, and engaging in rig

In [38]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.196.120.83
your url is: https://slick-moles-shine.loca.lt
